# Subreddit Classifier  
  
The social media platform Reddit is divided up into different subreddits. Each subreddit is a messageboard devoted to a certain topic. Each post contains a title and some combination of body text, images, videos, or links to external websites. People viewing the content can comment on it. Reddit uses a vote system where everybody can vote on each post or comment, either increasing the score (an "upvote") or decreasing the score (a "downvote") where comments or posts with higher scores generally being made more visible to anybody viewing the website. Some of the popular subreddit topics can include politics, news, humourous content, or AskReddit, a subreddit where people post questions and other people answer the questions in the comment section.

In order to scrape Reddit content, PRAW (the Python Reddit API Wrapper) was employed. It requires a Reddit developer account which gives credentials needed to access the API. Since I am only pulling content and not posting anything, I am accessing the API in read-only mode which requires fewer credentials than accessing it in read-write mode.  
  
The cell below handles imports and initializes PRAW with my credentials that are loaded via an external file.

In [1]:
#Imports and setting up the reddit API
import pandas as pd
import os
import praw
import numpy as np
import fasttext

#Client ID, secret, user agent in that order
credentials = list(pd.read_csv(os.getcwd()+'/credentials.csv').columns)

reddit = praw.Reddit(client_id=credentials[0],
                     client_secret=credentials[1],user_agent=credentials[2])

#I want:
#Title, text, link (if applicable), top comments, author


As a preliminary test, the cell below gathers the top 1000 posts (highest score) from six subreddits. These subreddits are:  
politics, a subreddit focused on US politics,  
AskReddit, a subreddit where people post questions and others answer them in the comment section,  
Worldnews, a subreddit focused on international, non-US news,  
Funny, a subreddit mostly for memes and joke content,  
Gaming, a subreddit dedicated to news or stories about videogames,  
Aww, a subreddit dedicated to photos and videos of animals, mostly pets.

In [32]:
#Gathers top posts from a few subreddits
politics = list(reddit.subreddit('politics').top(limit=1000))
ask = list(reddit.subreddit('askreddit').top(limit=1000))
worldnews = list(reddit.subreddit('worldnews').top(limit=1000))
funny = list(reddit.subreddit('funny').top(limit=1000))
gaming = list(reddit.subreddit('gaming').top(limit=1000))
aww = list(reddit.subreddit('aww').top(limit=1000))

The code above will gather lists of PRAW submission objects which then need to have the relevant data extracted from them.  
My general idea is to create a Pandas dataframe containing all of the relevant data. Since PRAW requires each submission be handled individually (I cannot just grab the list, type list.title and get a list of all of the titles), I will be creating empty lists and appending the relevant values to them while looping over my lists of submissions.  
  
For now, Reddit comments will be ignored.

In [43]:
#Initializing empty lists
titles = []
is_self = []
is_video = []
selftext = []
author = []
created = []
num_comments = []
score = []
subreddit = []
for i in range(len(politics)):
    titles.append(politics[i].title)
    is_self.append(politics[i].is_self)
    is_video.append(politics[i].is_video)
    selftext.append(politics[i].selftext)
    author.append(politics[i].author)
    created.append(politics[i].created)
    num_comments.append(politics[i].num_comments)
    score.append(politics[i].score)
    subreddit.append(0)
for i in range(len(ask)):
    titles.append(ask[i].title)
    is_self.append(ask[i].is_self)
    is_video.append(ask[i].is_video)
    selftext.append(ask[i].selftext)
    author.append(ask[i].author)
    created.append(ask[i].created)
    num_comments.append(ask[i].num_comments)
    score.append(ask[i].score)
    subreddit.append(1)
for i in range(len(worldnews)):
    titles.append(worldnews[i].title)
    is_self.append(worldnews[i].is_self)
    is_video.append(worldnews[i].is_video)
    selftext.append(worldnews[i].selftext)
    author.append(worldnews[i].author)
    created.append(worldnews[i].created)
    num_comments.append(worldnews[i].num_comments)
    score.append(worldnews[i].score)
    subreddit.append(2)
for i in range(len(funny)):
    titles.append(funny[i].title)
    is_self.append(funny[i].is_self)
    is_video.append(funny[i].is_video)
    selftext.append(funny[i].selftext)
    author.append(funny[i].author)
    created.append(funny[i].created)
    num_comments.append(funny[i].num_comments)
    score.append(funny[i].score)
    subreddit.append(3)
for i in range(len(gaming)):
    titles.append(gaming[i].title)
    is_self.append(gaming[i].is_self)
    is_video.append(gaming[i].is_video)
    selftext.append(gaming[i].selftext)
    author.append(gaming[i].author)
    created.append(gaming[i].created)
    num_comments.append(gaming[i].num_comments)
    score.append(gaming[i].score)
    subreddit.append(4)
for i in range(len(aww)):
    titles.append(aww[i].title)
    is_self.append(aww[i].is_self)
    is_video.append(aww[i].is_video)
    selftext.append(aww[i].selftext)
    author.append(aww[i].author)
    created.append(aww[i].created)
    num_comments.append(aww[i].num_comments)
    score.append(aww[i].score)
    subreddit.append(5)



In [44]:
#This creates a Pandas dataframe to hold the data
dat = pd.DataFrame()
dat['title'] = titles
dat['text'] = selftext
dat['author'] = author
dat['created'] = created
dat['score'] = score
dat['num_comments'] = num_comments
dat['is_self'] = is_self
dat['is_video'] = is_video
dat['subreddit'] = subreddit
dat = dat.sample(frac=1).reset_index(drop=True)

In [45]:
#Here, short titles are dropped since they are unlikely to have much predictive power 
from nltk.tokenize import word_tokenize
short_count = 0
for i in range(len(dat)):
    if (len(word_tokenize(dat['title'][i]))<2):
        short_count = short_count + 1
        dat = dat.drop(i,axis='index')
print(str(short_count)+' titles removed')
#Resets the index of the dataframe due to the dropped frames
dat = dat.reset_index(drop=True)

44 titles removed


Now that the data is wrangled into a format that is easier to use, I am just going to shove it into the fasttext library Facebook keeps on their research Github. It might not work that well, but it'll at least be a quick prototype.

First, the fasttext package direct from Facebook's Github will be employed, mostly because using it is pretty simple and does not really require the normal NLP preprocessing.  
  
It can be trained in supervised mode which supplies a classifier, though it requires the data be loaded from an external text file where each entry looks like this:  
  
\__label__*thelabel* The text  
  
Where "*thelabel*" is the label of that data point and "The text" is the actual text. The part "\__label__" needs to be  placed as is to mark what the label is.

In [46]:
#Splits into training and test data
train = dat[0:int(len(dat)*.9)]
test = dat[int(len(dat)*.9):len(dat)].reset_index(drop=True)
datafile = []
#Generates data file for output based on training data
for i in range(len(train)):
    datafile.append('__label__'+str(train['subreddit'][i])+' '+train['title'][i])

In [47]:
#This saves the data as a text file as per fasttext's requirements
#this opens the data file
outfile = open('data.txt','w',errors='ignore')
#This runs through the data and adds each line
for line in datafile:
    outfile.write(line)
    outfile.write('\n')
outfile.close()

In [48]:
#This trains the fasttext model
model = fasttext.train_supervised('data.txt')

In [49]:
#This runs all of the training and testing examples through the fasttext classifier
trainout = []
for i in range(len(train)):
    trainout.append(int(model.predict(train['title'][i])[0][0][9]))
testout = []
for i in range(len(test)):
    testout.append(int(model.predict(test['title'][i])[0][0][9]))

In [50]:
#This counts up the training and test error
trainacc = 0
#loops over training data
for i in range(len(train)):
    #If the model's output is different from the 
    if (train['subreddit'][i]!=trainout[i]):
        trainacc = trainacc + 1
trainacc = 1- trainacc/len(trainout)

testacc = 0
for i in range(len(test)):
    if (test['subreddit'][i] != testout[i]):
        testacc = testacc + 1
testacc = 1-testacc/len(testout)
print('The training set accuracy is: '+str(trainacc))
print('The test set accuracy is: '+str(testacc))

The training set accuracy is: 0.6068745304282495
The test set accuracy is: 0.5591216216216216


In [51]:
#This uses sklearn to plot the confusion matrix for the test set, just to see how it looks
from sklearn.metrics import confusion_matrix

print('politics','askreddit','worldnews','funny','gaming','aww')
confusion = pd.DataFrame(confusion_matrix(test['subreddit'],testout,normalize='true'))
print(confusion)
#predicted along x-axis, true along y-axis

politics askreddit worldnews funny gaming aww
          0         1         2         3         4         5
0  0.847619  0.000000  0.076190  0.047619  0.019048  0.009524
1  0.000000  0.959596  0.030303  0.000000  0.010101  0.000000
2  0.514019  0.009346  0.392523  0.028037  0.056075  0.000000
3  0.021978  0.000000  0.021978  0.241758  0.296703  0.417582
4  0.051546  0.061856  0.041237  0.268041  0.309278  0.268041
5  0.010753  0.032258  0.032258  0.193548  0.161290  0.569892


The confusion matrix actually shows some pretty interesting results. The highest accuracy rating goes to AskReddit where it correctly predicts it 95% of the time while also rarely falsely predicting askreddit.

That didn't work very well. Maybe it's the classifier, instead of using the supervised fasttext model, I will try the unsupervised fasttext embeddings and then use boosting

In [12]:
import gensim.downloader as api
dataset = api.load('text8')
c = list(dataset)
from gensim.models import FastText
unsupftt = FastText(size=40)
unsupftt.build_vocab(c)

#model = fasttext.load_model(os.getcwd()+'/fasttextmodel.vec')

In [13]:
unsupftt.train(c,total_examples=unsupftt.corpus_count,epochs=3)

In [54]:
from nltk.tokenize import word_tokenize
embeddings = np.zeros((40,len(dat)))
for i in range(len(dat)):
    tokenized = word_tokenize(dat['title'][i])
    avg = np.zeros((1,40))
    for j in range(len(tokenized)):
        avg = avg + unsupftt.wv.get_vector(tokenized[j])
    avg = avg/len(tokenized)
    embeddings[:,i] = avg
    
embeddings = embeddings.T

In [55]:
import xgboost as xg
trainx = embeddings[0:5500]
testx = embeddings[5501:]
trainy = dat['subreddit'][0:5500]
testy = dat['subreddit'][5501:]
model = xg.XGBClassifier(gamma=3,subsample=0.8)
model.fit(trainx,trainy)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=3, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)

In [56]:
#With the trained model, show accuracy and the confusion matrix
from sklearn.metrics import confusion_matrix
tfidftrain = model.predict(trainx)
tfidfout = model.predict(testx)
confusion2 = pd.DataFrame(confusion_matrix(testy,tfidfout,normalize='true'))
print('Training set accuracy:')
print(sum(trainy==tfidftrain)/len(tfidftrain))
print('Test set accuracy:')
print(sum(testy==tfidfout)/len(tfidfout))
print(confusion2)

Training set accuracy:
0.9312727272727273
Test set accuracy:
0.4819277108433735
          0         1         2         3         4         5
0  0.552632  0.065789  0.302632  0.000000  0.039474  0.039474
1  0.028571  0.728571  0.100000  0.028571  0.071429  0.042857
2  0.250000  0.041667  0.625000  0.055556  0.013889  0.013889
3  0.057143  0.057143  0.071429  0.242857  0.257143  0.314286
4  0.130435  0.115942  0.115942  0.130435  0.289855  0.217391
5  0.017241  0.155172  0.051724  0.224138  0.120690  0.431034


That didn't work that well, maybe it's the Fasttext embeddings, though the data up until now has barely been cleaned or processed, which is typically important in natural language applications. I will start with the following: 
  
- Word Tokenization (splitting the strings into a list of one-word strings)
- Removal of stopwords
- Lemmatization or stemming
- Moving to a vector representation (bag of words, word2vec, or fasttext)

In [57]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stop = set(stopwords.words('english'))
titles = []
#This tokenizes, removes stopwords and stems
for i in range(len(dat)):
    #tokenizes by word
    tmp_title = word_tokenize(dat['title'][i].lower())
    #removes stop words
    tmp_title2 = [i for i in tmp_title if i not in stop]
    #stems
    tmp_title3 = [ps.stem(i) for i in tmp_title2]
    titles.append(tmp_title3)

Sklearn's TF-IDF vectorizer actually seems to do a lot of the work for you, so I'll give that a try

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=((1,3)))
x = vectorizer.fit_transform(dat['title'])

In [76]:
import xgboost as xg
trainx = x[0:5500]
testx = x[5501:]
trainy = dat['subreddit'][0:5500]
testy = dat['subreddit'][5501:]
model = xg.XGBClassifier(gamma=0.3,subsample=0.8)
model.fit(trainx,trainy)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.3, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)

In [77]:
#With the trained model, show accuracy and the confusion matrix
tfidftrain = model.predict(trainx)
tfidfout = model.predict(testx)
confusion2 = pd.DataFrame(confusion_matrix(testy,tfidfout,normalize='true'))
print('Training set accuracy:')
print(sum(trainy==tfidftrain)/len(tfidftrain))
print('Test set accuracy:')
print(sum(testy==tfidfout)/len(tfidfout))
print(confusion2)

Training set accuracy:
0.9169090909090909
Test set accuracy:
0.6650602409638554
          0         1         2         3         4         5
0  0.776316  0.000000  0.131579  0.052632  0.026316  0.013158
1  0.014286  0.928571  0.042857  0.000000  0.014286  0.000000
2  0.194444  0.000000  0.666667  0.083333  0.041667  0.013889
3  0.000000  0.014286  0.028571  0.557143  0.228571  0.171429
4  0.000000  0.028986  0.101449  0.333333  0.463768  0.072464
5  0.000000  0.017241  0.051724  0.206897  0.155172  0.568966


This result is a little surprising, tf-idf actually seems to work better than FastText here, both in having a greater overall test set accuracy and in resolving some of the issues with the confusion matrix. Its ability to tell apart the subreddits "funny" and "gaming," while still not stellar, has notably improved from FastText. This could be partly because the titles are typically quite short which would make word embeddings lose some of their power. 

In [78]:
from sklearn.naive_bayes import MultinomialNB

MnNB = MultinomialNB()
MnNB.fit(trainx,trainy)
MnNBtrain = MnNB.predict(trainx)
MnNBtest = MnNB.predict(testx)
confusion3 = pd.DataFrame(confusion_matrix(testy,MnNBtest,normalize='true'))
print('Training set accuracy:')
print(sum(trainy==MnNBtrain)/len(MnNBtrain))
print('Test set accuracy:')
print(sum(testy==MnNBtest)/len(MnNBtest))
print(confusion3)

Training set accuracy:
0.9847272727272727
Test set accuracy:
0.6506024096385542
          0         1         2         3         4         5
0  0.736842  0.039474  0.210526  0.000000  0.013158  0.000000
1  0.014286  0.985714  0.000000  0.000000  0.000000  0.000000
2  0.152778  0.000000  0.833333  0.000000  0.000000  0.013889
3  0.000000  0.128571  0.014286  0.157143  0.228571  0.471429
4  0.014493  0.231884  0.028986  0.072464  0.434783  0.217391
5  0.017241  0.155172  0.000000  0.017241  0.051724  0.758621


Looking at the confusion matrices for XGBoost vs Naive Bayes, they both have distinct strengths and weaknesses. Naive Bayes seems to do better for the classes that are not 3 and 4, whereas XGBoost does a better job of separating 3 and 4. Somehow combining the two classifiers might work well, but Naive Bayes generally seems biased against predicting 3 and 4, especially 3 which it predicts very infrequently. It also predicts the "more accurate" classes a lot more often, with classes 1 and 5 being predicted far more often than they actually appear in the data.